生成对抗网络-GAN(Generative Adversarial Networks)
===

# 1.概述
生成对抗网络可以分为普通GAN、DCGAN、CGAN、infoGAN、Wasserstein GAN、Wasserstein GAN-GP、LSGAN、BEGAN、PIX2PIX、CycleGAN和DiscoGAN等等。简单来说就是以下三点
- 构建两个网络：一个生成网络$G$，一个判别网络$D$，基本上两个网络都是需要使用到卷积
- 训练方式：判别网络接收来自生成网络生成的数据，然后判断真假，生成网络生成数据，尽量迷惑判别网络，一般来说G训练一次，D训练K次
- 数据输入：G网络的输入是Noise，D网络的输入是混合G网络的输出数据以及样本数据。G网络的目标是尽量生成逼真的假数据，使得D网络无法判别出入的图片是真还是假；D网络的目标是尽量的能够区分输入的图片是真还是假。这就是所谓的对抗过程

在下面的介绍中，我们假定真实样本空间为$X=\{x_1,x_2,...,x_n\}$, 噪声为$Z$，生成网络为$G$，判别网络为$D$

# 2.生成对抗网络-GAN(Generative Adversarial Networks)
## 2.1.概述
不使用卷积层的GAN是普通的生成对抗网络

## 2.2.网络结构
### 2.2.1.G网络
使用全连接层，RELU或者Leaky_Relu以及Sigmoid来生成数据

### 2.2.2.D网络
使用全连接层，RELU或者Leaky_Relu以及Sigmoid来判别数据

## 2.3.损失函数
$$\begin{eqnarray}
L_D^{GAN}&=&\frac{1}{n}\sum_{i=1}^n[log(D(x^i))]+E[log(1-D(G(z^i)))]\\
L_G^{GAN}&=&E[log(D(G(z)))]
\end{eqnarray}$$
优化目标是
$$min_Gmax_DLoss$$

# 3.深度卷积生成对抗网络-DCGAN(Deep Convolution Generative Adversarial Networks)
改进的就是在G和D网络中使用了Conv和Batch Norm层，G网络使用Tanh代替Sigmoid。<br/>
![images](../images/02_06_01_001.png)

# 4.条件生成对抗网络-CGAN(Conditional Generative Adversarial Networks)
## 4.1.概述
GAN中输入是随机的数据，没有太多意义，那么我们很自然的会想到能否用输入改成一个有意义的数据，最简单的就是数字字体生成，能否输入一个数字，然后输出对应的字体。这就是CGAN要做的事，就是在G网络的输入在z的基础上连接一个输入y，然后在D网络的输入在x的基础上也连接一个y。这样GAN就变成了有监督了<br/>
![images](../images/02_06_01_002.png)

# 5.信息生成对抗网络-infoGAN(Info Generative Adversarial Networks)
有了CGAN，我们可以有一个单一输入y，然后通过调整z输出不同的图像。但是CGAN是有监督的，我们需要指定y。那么有没有可能实现无监督的CGAN？这个想法本身就比较疯狂，要实现无监督的CGAN，意味着需要让神经网络不但通过学习提取了特征，还需要把特征表达出来。对于MNIST，如何通过无监督学习让神经网络知道你输入y=2时就输出2的字体？或者用一个连续的值来调整字的粗细，方向？感觉确实是一个非常困难的问题，但是InfoGAN就这么神奇的做到了。怎么做呢？作者引入了信息论的知识，也就是mutual information互信息。作者的思路就是G网络的输入除了z之外同样类似CGAN输入一个c变量，这个变量一开始神经网络并不知道是什么含义，但是没关系，我们希望c与G网络输出的x之间的互信息最大化，也就是让神经网络自己去训练c与输出之间的关系。

# 6.Wassertein生成对抗网络-WGAN(Wassertein Generative Adversarial Networks)
## 6.1.改进
1. 判别器最后一层去掉sigmoid
2. 生成器和判别器的loss不取log
3. 对更新后的权重强制截断到一定范围内，比如[-0.01，0.01]，以满足论文中提到的lipschitz连续性条件。
4. 论文中也推荐使用SGD， RMSprop等优化器，不要基于使用动量的优化算法，比如adam

在WGAN中，D的任务不再是尽力区分生成样本与真实样本，而是尽量拟合出样本间的Wasserstein距离，从分类任务转化成回归任务。而G的任务则变成了尽力缩短样本间的Wasserstein距离

## 6.2.WGAN的作用
1. WGAN理论上给出了GAN训练不稳定的原因，即交叉熵(JS散度)不适合衡量具有不相交部分的分布之间的距离，转而使用wassertein距离去衡量生成数据分布和真实数据分布之间的距离，理论上解决了训练不稳定的问题。
2. 解决了模式崩溃的(collapse mode)问题，生成结果多样性更丰富。
3. 对GAN的训练提供了一个指标，此指标数值越小，表示GAN训练的越差，反之越好。可以说之前训练GAN完全就和买彩票一样，训练好了算你中奖，没中奖也不要气馁，多买几注吧。

## 6.3.Wassertein距离
GAN中交叉熵(JS散度)不适合衡量生成数据分布和真实数据分布的距离，如果通过优化JS散度训练GAN会导致找不到正确的优化目标，所以，WGAN提出使用wassertein距离作为优化方式训练GAN，但是数学上和真正代码实现上还是有区别的，使用Wasserteion距离需要满足很强的连续性条件—lipschitz连续性，为了满足这个条件，作者使用了将权重限制到一个范围的方式强制满足lipschitz连续性，但是这也造成了隐患.Lipschitz限制是在样本空间中，要求判别器函数D(x)梯度值不大于一个有限的常数K，通过权重值限制的方式保证了权重参数的有界性，间接限制了其梯度信息

# 7.Wassertein增强型生成对抗网络(Wassertein Generative Adversarial Networks-Gradient Penalty)
WGAN-GP是WGAN之后的改进版，主要还是改进了连续性限制的条件，因为，作者也发现将权重剪切到一定范围之后，比如剪切到[-0.01,+0.01]后，发生了这样的情况，如下图左边表示<br/>
![images](../images/02_06_01_003.png)<br/>
发现大多数的权重都在-0.01 和0.01上，这就意味了网络的大部分权重只有两个可能数，对于深度神经网络来说不能充分发挥深度神经网络的拟合能力，简直是极大的浪费。并且，也发现强制剪切权重容易导致梯度消失或者梯度爆炸，梯度消失很好理解，就是权重得不到更新信息，梯度爆炸就是更新过猛了，权重每次更新都变化很大，很容易导致训练不稳定。梯度消失与梯度爆炸原因均在于剪切范围的选择，选择过小的话会导致梯度消失，如果设得稍微大了一点，每经过一层网络，梯度变大一点点，多层之后就会发生梯度爆炸 。为了解决这个问题，并且找一个合适的方式满足lipschitz连续性条件，作者提出了使用梯度惩罚(gradient penalty)的方式以满足此连续性条件，其结果如上图右边所示.由于是对每个batch中的每一个样本都做了梯度惩罚（随机数的维度是(batchsize，1)），因此判别器中不能使用batch norm,但是可以使用其他的normalization方法，比如Layer Normalization、Weight Normalization和Instance Normalization，但是训练结果中WGAN-GP生成的图片比较模糊

# 8.最小二乘生成对抗网络-LSGAN(Least Squares Generative Adversarial Networks)
LSGAN的原理其实就是使用了最小二乘损失函数代替了GAN的损失函数，但是就这样，缓解了GAN训练不稳定和生成图像质量差、多样性不足的问题.使用JS散度并不能拉近真实分布和生成分布之间的距离，使用最小二乘可以将图像的分布尽可能的接近决策边界

# 9.边界平衡生成对抗网络-BEGAN(Boundary Equilibrium Generative Adversarial Networks)
## 9.1.主要贡献
1. 提出了一种新的简单强大GAN，使用标准的训练方式，不加训练trick也能很快且稳定的收敛
2. 对于GAN中G，D的能力的平衡提出了一种均衡的概念（GAN的理论基础就是goodfellow理论上证明了GAN均衡点的存在，但是一直没有一个准确的衡量指标说明GAN的均衡程度）
3. 提出了一种收敛程度的估计，这个机制只在WGAN中出现过。作者在论文中也提到，他们的灵感来自于WGAN，在此之前只有wgan做到了

## 9.2.传统的GAN原理
BEGAN的判别器使用的是auto-encoder结构，输入的是图片，输出的是经过编码解码后的图片。以往的GAN及其变种都是希望生成器生成的数据分布尽可能的接近真实数据的分布，当生成数据分布等同于真是数据分布时，我们就确定生成器G经过训练可以生成和真是数据分布相同的样本，即获得了生成足以以假乱真数据的能力，以从这一点出发，研究者们设计了各种损失函数去令G的生成数据分布尽可能接近真实数据分布

## 9.3.BEGAN的原理
BEGAN代替了这种估计概率分布方法，它不直接去估计生成分布$Pg$与真实分布$Px$的差距，进而设计合理的损失函数拉近他们之间的距离，而是估计分布的误差之间的距离，作者认为只要分布的的误差分布相近的话，也可以认为这些分布是相近的，也就是说如果我们认为A和B很相似，而B和C也很相似，那么我们就认为A和C也很相似。在训练中，A相当于训练数据$X$,B相当于判别器$D$对于数据$X$编码解码后的图像$D(X)$，C就相当于$D$以$G$的生成未输入的结果$D[G(z)]$，所以，如果$||D(X)-X||-||D(X)-D[G(z)]||$不断趋近于0，那么随着训练，$D(X)$就会不断的趋近于$X$,那么$D[G(z)]$就会接近于$D(X)$，那么岂不是就意味着$G(z)$的数据分布和$X$的分布几乎一样了，那么就说明G学到了生成数据的能力。